In [37]:
import os, signal, rospy
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from time import sleep, time
from shutil import get_terminal_size
from Simulator.src.automobile_data_simulator import AutomobileDataSimulator
from Simulator.src.helper_functions import *
from path_nn_controller import PathPlanning, Controller, Detection

In [38]:
LAPS = 3
imgs = []
locs = []

In [39]:
REVERSED_PATH = False #if True the path is Anti-clockwise
STEER_NOISE_STD = np.deg2rad(0.0) # [rad] noise in the steering angle
STEER_FRAME_CHAMGE_MEAN = 10 #avg frames after which the steering noise is changed
STEER_FRAME_CHAMGE_STD = 8 #frames max "deviation"
CONTROLLER_DIST_AHEAD = .35 #pure pursuit controller distance ahead
DESIRED_SPEED = 1.5# [m/s]
TARGET_FPS = 30.0

#initializations
os.system('rosservice call /gazebo/reset_simulation') 
CAR = AutomobileDataSimulator(trig_cam=True, trig_gps=True, trig_bno=True, 
                               trig_enc=True, trig_control=True, trig_estimation=False, trig_sonar=False)

CONTROLLER = Controller(1.0)

STEER_NOISE = MyRandomGenerator(0.0, STEER_NOISE_STD, STEER_FRAME_CHAMGE_MEAN, STEER_FRAME_CHAMGE_STD)
SPEED_NOISE_STD = 0.0  #[m/s] noise in the speed
SPEED_FRAME_CHAMGE_MEAN = 30 #avg frames after which the speed noise is changed
SPEED_FRAME_CHAMGE_STD = 20 #frames max "deviation"
SPEED_NOISE = MyRandomGenerator(-SPEED_NOISE_STD, SPEED_NOISE_STD, SPEED_FRAME_CHAMGE_MEAN, SPEED_FRAME_CHAMGE_STD, np.random.uniform)

sleep(0.8)

LAP_Y_TRSH = 2.54 + 2.5

Exception in thread Thread-39:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-40:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-41:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 228, in run
    self.run()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 228, in run
    self.run()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 228, in run
    r.sleep()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 103, in sleep
    r.sleep()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 103, in sleep
    r.sleep()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 103

In [40]:
path = np.load('sparcs/sparcs_path_precise.npy').T # load path from file
#add 2 to all x
path[:,0] += 2.5
path[:,1] += 2.5

cv.namedWindow('frame', cv.WINDOW_NORMAL)

#lap logic
lap = 0
prev_y = CAR.y_true

while not rospy.is_shutdown():
    loop_start = time()

    x, y, yaw = CAR.x_true, CAR.y_true, CAR.yaw
    frame = CAR.frame

    locs.append(np.array([x, y, yaw]))

    tmp_frame = frame.copy()
    
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame = cv.resize(frame, (320, 240), interpolation=cv.INTER_AREA)
    imgs.append(frame)

    #heading error
    he, pa, d = get_heading_error(x,y,yaw,path,CONTROLLER_DIST_AHEAD)

    #calculate control
    steer_angle = CONTROLLER.get_control(he, CONTROLLER_DIST_AHEAD)
    steer_angle = steer_angle + STEER_NOISE.get_noise()

    #move car
    CAR.drive(speed=DESIRED_SPEED, angle=np.rad2deg(steer_angle))

    cv.imshow('frame', tmp_frame)
    if cv.waitKey(1) == 27:
        CAR.stop()
        break

    #laps
    curr_y = y
    if (prev_y > LAP_Y_TRSH and curr_y < LAP_Y_TRSH and not REVERSED_PATH) or (prev_y < LAP_Y_TRSH and curr_y > LAP_Y_TRSH and REVERSED_PATH): #count lap
        lap+=1
        if lap>=LAPS:
            # saving locations
            #stop the car
            print('Stopping')
            CAR.stop()
            break
    prev_y = curr_y



    #calculate fps
    loop_time = time() - loop_start
    fps = 1.0 / loop_time

    print(f'x: {x:.2f}, y: {y:.2f}, yaw: {np.rad2deg(yaw):.2f}, fps: {fps:.2f}, lap: {lap+1}/{LAPS}', end='\r')

    if loop_time < 1/TARGET_FPS:
        sleep(1/TARGET_FPS - loop_time)

cv.destroyAllWindows()


Stopping y: 5.05, yaw: -90.00, fps: 232.82, lap: 3/33
